In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer

In [2]:
df_jan = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df_feb = pd.read_parquet('yellow_tripdata_2022-02.parquet')

In [3]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [4]:
df_feb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979431 entries, 0 to 2979430
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Q1

Answer:

In [5]:
print(len(df_jan.columns))

19


## Q2

In [6]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [7]:
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
df_jan["duration"].dtype

dtype('<m8[ns]')

In [8]:
df_jan["duration"] = df_jan["duration"].dt.total_seconds()/60

Answer:

In [9]:
df_jan["duration"].std()

46.44530513776802

## Q3

In [10]:
len_all_durations = len(df_jan["duration"])
filter_durations = (df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)
len_filtered_durations = len(df_jan[filter_durations])

Answer:

In [11]:
print(len_all_durations)
print(len_filtered_durations)
print(len_filtered_durations/len_all_durations)

2463931
2421440
0.9827547930522406


In [12]:
df_jan = df_jan[filter_durations]

## Q4

In [13]:
ids_loc_df  = df_jan[["PULocationID", "DOLocationID"]].fillna(-1).astype('int')
ids_loc_df.head()

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166
3,114,68
4,68,163


In [14]:
ids_loc_df = ids_loc_df.astype('str')
train_dicts = ids_loc_df.to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts) 

Answer:

In [15]:
X_train.shape

(2421440, 515)

## Q5

In [16]:
y_train = df_jan.duration.values

In [17]:

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [18]:
y_pred = model.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986190685619784

Answer: 6.99

## Q6

In [19]:
def get_val_data(df, dv=dv):
    df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    ids_loc_df  = df[["PULocationID", "DOLocationID"]].fillna(-1).astype('int').astype('str')
    
    val_dicts = ids_loc_df.to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    y_val = df.duration.values
    
    try:
        assert X_val.shape[0] == y_val.shape[0]
    except AssertionError:
        print(f"ERROR: {X_val.shape[0]} != {y_val.shape[0]}")
        return None, None
        
    return X_val, y_val

In [20]:
X_val, y_val = get_val_data(df_feb)

In [21]:
y_pred = model.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.786409438115243

Answer: 7.79